**Load data as obtained from the Microplate reader and build a dataframe with the raw values**

Run this chunk to generate a dataframe (df) with raw values

In [ ]:
import pandas as pd

# Treatment labels (16 treatments × 4 replicates)
treatments = [
    "NC", "Dip4", "Dip8", "Dip16",
    "T-360", "T-180", "T-90",
    "Dip4/T-360", "Dip4/T-180", "Dip4/T-90",
    "Dip8/T-360", "Dip8/T-180", "Dip8/T-90",
    "Dip16/T-360", "Dip16/T-180", "Dip16/T-90"
]

# Load the 96 wells plate data
df_plate = pd.read_csv("dip_tuc_1.csv", header=None, skiprows=45, sep= ",")


# Drop first column (A-H) and replace "EP" with NA
df_values = df_plate.drop(columns=[0]).astype(str).replace(r'^\s*EP\s*$', pd.NA, regex=True)

# Initialize result container
data = []

# Current treatment index
treatment_index = 0

# Loop over columns
for col in range(df_values.shape[1]):
    col_values = df_values.iloc[:, col]

    # Extract all valid (non-na) values from the column
    valid_vals = col_values.dropna().tolist()

    # Process in groups of 4 (replicates)
    for i in range(0, len(valid_vals), 4):
        reps = valid_vals[i:i+4]
        if len(reps) == 4 and treatment_index < len(treatments):
            data.append([treatments[treatment_index]] + reps)
            treatment_index += 1

# Build DataFrame with raw data
df_raw = pd.DataFrame(data, columns=["treatment", "rep1", "rep2", "rep3", "rep4"])

#save raw data. If needed, uncomment the next line
#df_raw.to_csv("df_raw.csv", index=False)


**Build a dataframe adjusted by NC**

Run the previous chunk to generate the raw data frame and this chunk to generate a negative control adjusted dataframe. This df includes mean and standard deviation. The reads are expressed as viability (%).
You can use this df to generate a bar plot with error bars using the R script "Drug screenning".

In [ ]:
import numpy as np

# Rename for clarity and to keep both files
df_plot = df_raw.copy()

# Calculate mean of the negative control (NC)
nc_values = df_plot[df_plot["treatment"] == "NC"].iloc[0, 1:5].values.astype(float)
nc_mean = np.mean(nc_values)

# Normalize OD values to viability
for rep in ["rep1", "rep2", "rep3", "rep4"]:
    df_plot[rep] = df_plot[rep].astype(float) / nc_mean * 100

# Add 'mean' and 'SD' columns
df_plot["mean"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].mean(axis=1)
df_plot["SD"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].std(axis=1)

# Preview
print(df_plot)

# Save the normalized data to plot
df_plot.to_csv("df_plot.csv", index=False)


**Built DataFrame for SynergyFinder**

Run this chunk to generate a df to calculate and visualize Synergy Scores for Drug Combinations using the "Drug screening" script in R

In [35]:
import pandas as pd
import numpy as np

# Define all treatment combinations
treatments = [
    (0, 0), (4, 0), (8, 0), (16, 0),
    (0, 360), (0, 180), (0, 90),
    (4, 360), (4, 180), (4, 90),
    (8, 360), (8, 180), (8, 90),
    (16, 360), (16, 180), (16, 90)
]

# Load the 96 wells plate data
df_plate = pd.read_csv("dip_tuc_1.csv", header=None, skiprows=45, sep= ",")
# Drop first column (A-H) and replace "EP" with NA
df_values = df_plate.drop(columns=[0]).astype(str).replace(r'^\s*EP\s*$', pd.NA, regex=True)


# Calculate mean of the negative control (NC)
nc_values = df_values.iloc[:4, 0].astype(str).str.replace(",", ".").astype(float).tolist()
nc_mean = np.mean(nc_values)

# Initialize result container
data_rows = []

# Current treatment index
treatment_index = 0

# Loop over columns
for col in range(df_values.shape[1]):
    col_values = df_values.iloc[:, col]
    
    # Extract all valid (non-NA) values from the column
    valid_vals = col_values.dropna().tolist()
    
    # Process in groups of 4 (replicates)
    for i in range(0, len(valid_vals), 4):
        reps = valid_vals[i:i+4]
        if len(reps) == 4 and treatment_index < len(treatments):
            conc1, conc2 = treatments[treatment_index]
            # Convert values to float and normalize
            normalized_reps = [(float(rep.replace(",", ".")) / nc_mean * 100) for rep in reps]
            
            data_rows.append({
                "block_id": 1,
                "drug1": "Dipyridamole",
                "drug2": "Tucidinostat",
                "conc1": conc1,
                "conc2": conc2,
                "conc_unit1": "uM",
                "conc_unit2": "nM",
                "rep1": normalized_reps[0],
                "rep2": normalized_reps[1],
                "rep3": normalized_reps[2],
                "rep4": normalized_reps[3]
            })
            treatment_index += 1

# Build DataFrame
df_synergy = pd.DataFrame(data_rows)

# Convert from wide to long format and drop the replicate column
df_synergy = pd.melt(
    df_synergy,
    id_vars=["block_id", "drug1", "drug2", "conc1", "conc2", "conc_unit1", "conc_unit2"],
    value_vars=["rep1", "rep2", "rep3", "rep4"],
    value_name="response"
).drop(columns="variable")  # This removes the column with identifiers used to melt

# Save to CSV
df_synergy.to_csv("synergyfinder_df.csv", index=False)